# Past Race History Generator (Cross-Domain)

- **目的**: 過去走データを自己結合で生成（JRA/NAR両方を参照）
- **特徴**:
  - 交流競走・転籍馬に対応
  - JRA馬のNAR出走履歴、NAR馬のJRA出走履歴も取得
  - HTTPリクエスト不要
- **出力**: `database_xxx_with_history.csv`


In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ========================================
# ⚙️ モード設定 - ここを変更してください
# ========================================

MODE = 'JRA'  # 'JRA' または 'NAR' を選択

# ========================================

In [ ]:
# Settings
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw'

# 両方のファイルを読み込む（過去走検索用）
JRA_BASIC = 'database_jra_basic.csv'
NAR_BASIC = 'database_nar_basic.csv'

# 出力ファイル（モードに基づく）
if MODE == 'JRA':
    TARGET_FILE = JRA_BASIC
    OUTPUT_FILE = 'database_jra_with_history.csv'
elif MODE == 'NAR':
    TARGET_FILE = NAR_BASIC
    OUTPUT_FILE = 'database_nar_with_history.csv'
else:
    raise ValueError(f"Invalid MODE: {MODE}")

print(f"📋 Mode: {MODE}")
print(f"   Target races: {TARGET_FILE}")
print(f"   History source: {JRA_BASIC} + {NAR_BASIC} (cross-domain)")
print(f"   Output: {OUTPUT_FILE}")

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm

print("✅ Libraries loaded")

In [ ]:
# Column mappings: Basic column -> past_N field name
COLUMN_MAPPING = {
    '日付': 'date',
    '着順': 'rank',
    'タイム': 'time',
    '騎手': 'jockey',
    '馬体重': 'horse_weight',
    '増減': 'weight_change',
    '天候': 'weather',
    '馬場状態': 'condition',
    '距離': 'distance',
    'コースタイプ': 'course_type',
    '後3F': 'last_3f',
    '単勝オッズ': 'odds',
    'レース名': 'race_name',
    '会場': 'venue',  # 追加: どの競馬場で走ったか
}

print(f"✅ Mapping {len(COLUMN_MAPPING)} fields per past race")

In [ ]:
# Main Processing Function

def generate_cross_domain_history():
    jra_path = os.path.join(SAVE_DIR, JRA_BASIC)
    nar_path = os.path.join(SAVE_DIR, NAR_BASIC)
    target_path = os.path.join(SAVE_DIR, TARGET_FILE)
    output_path = os.path.join(SAVE_DIR, OUTPUT_FILE)
    
    # ========================================
    # Step 1: Load and combine history source
    # ========================================
    print("📂 Loading data sources...")
    
    df_history_parts = []
    
    # Load JRA
    if os.path.exists(jra_path):
        df_jra = pd.read_csv(jra_path, dtype=str)
        df_jra['source'] = 'JRA'
        df_history_parts.append(df_jra)
        print(f"   JRA: {len(df_jra)} rows")
    else:
        print(f"   ⚠️ JRA file not found: {jra_path}")
    
    # Load NAR
    if os.path.exists(nar_path):
        df_nar = pd.read_csv(nar_path, dtype=str)
        df_nar['source'] = 'NAR'
        df_history_parts.append(df_nar)
        print(f"   NAR: {len(df_nar)} rows")
    else:
        print(f"   ⚠️ NAR file not found: {nar_path}")
    
    if not df_history_parts:
        print("❌ No data files found!")
        return None
    
    # Combine for history lookup
    df_all_history = pd.concat(df_history_parts, ignore_index=True)
    print(f"   Combined history source: {len(df_all_history)} rows")
    
    # ========================================
    # Step 2: Load target data
    # ========================================
    print(f"\n📋 Loading target ({MODE})...")
    if not os.path.exists(target_path):
        print(f"❌ Target file not found: {target_path}")
        return None
    
    df_target = pd.read_csv(target_path, dtype=str)
    print(f"   Target rows: {len(df_target)}")
    
    # ========================================
    # Step 3: Prepare data
    # ========================================
    # Convert dates
    df_all_history['date_obj'] = pd.to_datetime(df_all_history['日付'], errors='coerce')
    df_target['date_obj'] = pd.to_datetime(df_target['日付'], errors='coerce')
    
    # Sort history by horse and date
    df_all_history = df_all_history.sort_values(['horse_id', 'date_obj'])
    
    # Initialize past columns in target
    past_fields = list(COLUMN_MAPPING.values())
    for i in range(1, 6):
        for field in past_fields:
            df_target[f'past_{i}_{field}'] = ''
    
    # ========================================
    # Step 4: Generate cross-domain history
    # ========================================
    print(f"\n🔄 Generating cross-domain history for {MODE}...")
    
    # Group combined history by horse_id
    grouped = df_all_history.groupby('horse_id')
    
    cross_domain_count = 0
    
    for idx in tqdm(range(len(df_target)), desc="Processing"):
        row = df_target.iloc[idx]
        horse_id = row['horse_id']
        current_date = row['date_obj']
        
        if pd.isna(current_date) or pd.isna(horse_id):
            continue
        
        # Get all races for this horse (from BOTH JRA and NAR)
        try:
            horse_races = grouped.get_group(horse_id)
        except KeyError:
            continue
        
        # Filter to races before current date
        past_races = horse_races[horse_races['date_obj'] < current_date]
        
        if past_races.empty:
            continue
        
        # Sort by date descending and take top 5
        past_races = past_races.sort_values('date_obj', ascending=False).head(5)
        
        # Check if any past race is from the other domain
        current_source = 'JRA' if MODE == 'JRA' else 'NAR'
        if (past_races['source'] != current_source).any():
            cross_domain_count += 1
        
        # Fill past columns
        for i, (_, past_row) in enumerate(past_races.iterrows(), 1):
            for basic_col, field_name in COLUMN_MAPPING.items():
                if basic_col in past_row.index:
                    val = past_row[basic_col]
                    df_target.at[idx, f'past_{i}_{field_name}'] = str(val) if pd.notna(val) else ''
    
    # ========================================
    # Step 5: Save result
    # ========================================
    # Drop temporary columns
    df_target = df_target.drop(columns=['date_obj'])
    
    print(f"\n💾 Saving to {OUTPUT_FILE}...")
    df_target.to_csv(output_path, index=False)
    
    # Stats
    has_past1 = (df_target['past_1_date'] != '').sum()
    print(f"\n✅ Complete! ({MODE})")
    print(f"   Total rows: {len(df_target)}")
    print(f"   Rows with past_1 data: {has_past1} ({has_past1/len(df_target)*100:.1f}%)")
    print(f"   🔄 Cross-domain history: {cross_domain_count} rows have JRA↔NAR past races")
    
    return df_target

print("✅ Function ready")

In [ ]:
# 実行
df_result = generate_cross_domain_history()

In [ ]:
# 結果確認（クロスドメイン履歴のサンプル）
if df_result is not None:
    # 過去走に別ドメインの会場が含まれるケースを探す
    jra_venues = {'札幌', '函館', '福島', '新潟', '東京', '中山', '中京', '京都', '阪神', '小倉'}
    
    print("\n📊 Cross-domain history examples:")
    count = 0
    for _, row in df_result.iterrows():
        past_venue = row.get('past_1_venue', '')
        if past_venue:
            is_jra_venue = past_venue in jra_venues
            is_current_jra = MODE == 'JRA'
            
            # Cross-domain if venue doesn't match mode
            if (is_current_jra and not is_jra_venue) or (not is_current_jra and is_jra_venue):
                print(f"\n  Horse: {row.get('馬名', 'N/A')} ({row['horse_id']})")
                print(f"    Current ({MODE}): {row['日付']} @ {row.get('会場', 'N/A')}")
                print(f"    Past 1: {row['past_1_date']} @ {past_venue} (cross-domain!)")
                count += 1
                if count >= 3:
                    break
    
    if count == 0:
        print("  (No cross-domain examples found in sample)")